In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd

# Load and prepare your dataset
# Ensure your CSV has columns 'tweet' for the text and 'class' for the labels
data = pd.read_csv("abusedetection.csv")
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['tweet'].tolist(), data['class'].tolist(), test_size=0.2, random_state=42
)

# Load pre-trained RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Tokenize the dataset
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

# Convert to PyTorch dataset
class AbuseDetectionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = AbuseDetectionDataset(train_encodings, train_labels)
val_dataset = AbuseDetectionDataset(val_encodings, val_labels)

# Load pre-trained RoBERTa model for sequence classification
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=3)

# Define accuracy metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch",
)

# Trainer for fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train and evaluate the model
trainer.train()
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss


In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
import xgboost as xgb  # Importing XGBoost
from tqdm import tqdm  # For progress bar

# Load the dataset
data = pd.read_csv("Language Detection.csv")

# Separating the independent and dependent features
X = data["Text"]
y = data["Language"]

# Converting categorical variables to numerical
le = LabelEncoder()
y = le.fit_transform(y)

# Preprocessing the text
data_list = []
for text in X:
    text = re.sub(r'[!@#$(),"%^*?:;~0-9]', ' ', text)  # Remove symbols and numbers
    text = re.sub(r'[[]]', ' ', text)
    text = text.lower()  # Convert text to lowercase
    data_list.append(text)

# Creating bag of words using CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(data_list).toarray()

# Train-test splitting
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Progress tracking for XGBoost model
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42, verbosity=1)

# Define the base learners with XGBoost included
base_learners = [
    ('nb', MultinomialNB()),  # Naive Bayes
    ('svm', SVC(probability=True)),  # Support Vector Machine with probability
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),  # Random Forest
    ('xgb', xgb_clf)  # XGBoost with progress tracking
]

# Define the meta learner (Logistic Regression)
meta_learner = LogisticRegression()

# Create the stacking classifier
stacking_clf = StackingClassifier(estimators=base_learners, final_estimator=meta_learner)

# Track progress with tqdm
print("Training Stacking Classifier...")
with tqdm(total=100, desc="Progress", position=0, leave=True) as pbar:
    stacking_clf.fit(x_train, y_train)
    pbar.update(100)

# Make predictions on the test set
y_pred_stacking = stacking_clf.predict(x_test)

# Calculate accuracy
accuracy_stacking = accuracy_score(y_test, y_pred_stacking)
print(f"\nAccuracy of Stacking Classifier with XGBoost: {accuracy_stacking:.5f}")

# Optional: Confusion matrix for further analysis
conf_matrix = confusion_matrix(y_test, y_pred_stacking)
print("\nConfusion Matrix:")
print(conf_matrix)

<ipython-input-2-11fad76b26ab>:30: FutureWarning: Possible nested set at position 1
  text = re.sub(r'[[]]', ' ', text)


Training Stacking Classifier...


Progress:   0%|          | 0/100 [00:00<?, ?it/s]